# Wstęp do Sztucznej Inteligencji - rok akademicki 2022/2023

Przed rozpoczęciem pracy z notatnikiem zmień jego nazwę zgodnie z wzorem: `NrAlbumu_Nazwisko_Imie_PoprzedniaNazwa`.

Przed wysłaniem notatnika upewnij się, że rozwiązałeś wszystkie zadania/ćwiczenia.

# Temat: Wnioskowanie oparte na wiedzy niepewnej. Wnioskowanie bayesowskie
Zapoznaj się z treścią niniejszego notatnika czytając i wykonując go komórka po komórce. Wykonaj napotkane zadania/ćwiczenia.

In [1]:
import pymc as pm
import arviz as az
import numpy as np
print('Pymc version:', pm.__version__)
print('Arviz version:', az.__version__)
print('Numpy version:', np.__version__)

Pymc version: 5.10.4
Arviz version: 0.15.1
Numpy version: 1.25.2


## Zadanie 1  (obowiązkowe, 5pkt.)
Zapożyczone z ćwiczeń do wykładu [Arabas, Cichosz](http://wazniak.mimuw.edu.pl/index.php?title=Sztuczna_inteligencja/SI_%C4%86wiczenia_4)

### Zamodeluj i odpowiedz na pytania.

W śledztwie dotyczącym zabójstwa inspektor Bayes rozważa dwie hipotezy:

- $H_1$ główny podejrzany zabił,
- $H_2$ główny podejrzany nie zabił,

oraz następujące możliwe fakty:

- $E_1$ na miejscu zbrodni znaleziono odciski palców głównego podejrzanego,
- $E_2$ główny podejrzany nie ma alibi na czas popełnienia zabójstwa,
- $E_3$ główny podejrzany miał motyw zabicia ofiary,
- $E_4$ główny podejrzany był widziany w sądziedztwie miejsca, w którym mieszka nielegalny handlarz bronią,
- $E_5$ świadek zbrodni podał rysopis zabójcy nie pasujący do głównego podejrzanego.

Zależności między takimi faktami a hipotezami opisują następujące prawdopodobieństwa:

$P(E_1|H_1)=0.7,\qquad P(E_1|H_2)=0.3,$

$P(E_2|H_1)=0.8,\qquad P(E_2|H_2)=0.4,$

$P(E_3|H_1)=0.9,\qquad P(E_3|H_2)=0.5,$

$P(E_4|H_1)=0.4,\qquad P(E_4|H_2)=0.2,$

$P(E_5|H_1)=0.2,\qquad P(E_5|H_2)=0.4.$

__W którym przypadku prawdopodobieństwo popełnienia zabójstwa byłoby największe?__

1. Gdyby znaleziono na miejscu zbrodni jego odciski palców.
2. Gdyby stwierdzono, że nie miał alibi i miał motyw.
3. Gdyby znaleziono na miejscu zbrodni jego odciski palców oraz stwierdzono, że był widziany w sąsiedztwie miejsca, w którym mieszka nielegalny handlarz bronią, ale świadek zbrodni podał rysopis zabójcy nie pasujący do głównego podejrzanego.

### TWÓJ PROGRAM:

In [7]:
#YOUR CODE HERE

with pm.Model() as BayesModel:
  killer = pm.Bernoulli('killer', 0.5)

  killerPrints = pm.Deterministic('killerPrints', pm.math.switch(killer, 0.7, 0.3))
  killerNotAlibi = pm.Deterministic('killerNotAlibi', pm.math.switch(killer, 0.8, 0.4))
  killerMotive = pm.Deterministic('killerMotive', pm.math.switch(killer, 0.9, 0.5))
  killerSeen = pm.Deterministic('killerSeen', pm.math.switch(killer, 0.4, 0.2))
  killerNotSimilar = pm.Deterministic('killerNotSimilar', pm.math.switch(killer, 0.2, 0.4))

  trace = pm.sample(10000, chains = 1, return_inferencedata = True)
  trace.posterior['killer'].values

  pKillerPrints = (trace.posterior['killer'].values[0]*trace.posterior['killerPrints'].values[0]).sum()/trace.posterior['killerPrints'].values[0].sum()
  print('pKillerPrints:', pKillerPrints)

  pKillerAlibiMotive = (trace.posterior['killer'].values[0]*trace.posterior['killerNotAlibi'].values[0]*trace.posterior['killerMotive'].values[0]).sum()/(trace.posterior['killerNotAlibi'].values[0]*trace.posterior['killerMotive'].values[0]).sum()
  print('pKillerAlibiMotive:', pKillerAlibiMotive)

  pKillerPrintsSeenNotSimilar = (trace.posterior['killer'].values[0]*trace.posterior['killerPrints'].values[0]*trace.posterior['killerSeen'].values[0]*trace.posterior['killerNotSimilar'].values[0]).sum()/(trace.posterior['killerPrints'].values[0]*trace.posterior['killerSeen'].values[0]*trace.posterior['killerNotSimilar'].values[0]).sum()
  print('pKillerPrintsSeenNotSimilar:', pKillerPrintsSeenNotSimilar)

pKillerPrints: 0.6998319731156986
pKillerAlibiMotive: 0.7824725590133421
pKillerPrintsSeenNotSimilar: 0.6998319731156984


### ODPOWIEDŹ:

YOUR ANSWER HERE

## Zadanie 2  (obowiązkowe, 5pkt.)

### Zamodeluj i odpowiedz na pytania.
System alarmowy w mieszkaniu, reaguje na włamania oraz, niestety, również na drobne trzęsienia (ziemi). Sąsiedzi John i Mary są umówieni, żeby zadzwonić do właściciela gdy usłyszą alarm. John jest nadgorliwy i bierze różne zdarzenia (np. dzwonek telefonu) za sygnał alarmowy (i wtedy zawsze dzwoni). Mary rozpoznaje alarm poprawnie, lecz często słucha głośnej muzyki i może go w ogóle nie usłyszeć.

Sieć przekonań dla systemu alarmowego wygląda następująco:
![bsiec.PNG](http://torus.uck.pk.edu.pl/~amarsz/images/bsiec.PNG)

__Jakie jest prawdopodobieństwo, że:__
1. włączy się alarm?
2. doszło do włamanie jeśli wiadom, że włączył się alarm?
3. zdarzyło się trzęsienie ziemi jeśli wiadomo, żę włączył się alarm?
1. w razie włamania ktoś zadzwoni?
2. zawiadomienie o włamaniu jest fałszywe?
3. rozległ się alarm, przy czym nie wystąpiło ani trzęsienie ziemi ani włamanie, ale oboje John i Mary zadzwonili? (prawd. bezwarunkowe)

TWÓJ PROGRAM:

In [15]:
#YOUR CODE HERE

with pm.Model() as burglaryModel:
  burglary = pm.Bernoulli('burglary', 0.001)
  earthquake = pm.Bernoulli('earthquake', 0.002)

  pAlarm = pm.Deterministic('pAlarm', pm.math.switch(earthquake, pm.math.switch(burglary, 0.95, 0.29), pm.math.switch(burglary, 0.94, 0.001)))
  alarm = pm.Bernoulli('alarm', pAlarm)

  pJohn = pm.Deterministic('pJohn', pm.math.switch(alarm, 0.9, 0.05))
  john = pm.Bernoulli('john', pJohn)
  pMary = pm.Deterministic('pMary', pm.math.switch(alarm, 0.7, 0.01))
  mary = pm.Bernoulli('mary', pMary)

  trace = pm.sample(10000, chains=1, return_inferencedata=True)
  trace.posterior['alarm'].values

  print('pAlarm:', trace.posterior['pAlarm'].values[0].sum())

  pBurglaryAlarm = (trace.posterior['burglary'].values[0]*trace.posterior['alarm'].values[0]).sum()/trace.posterior['alarm'].values[0].sum()
  print('pBurglaryAlarm:', pBurglaryAlarm)

  pEarthquakeAlarm = (trace.posterior['earthquake'].values[0]*trace.posterior['alarm'].values[0]).sum()/trace.posterior['alarm'].values[0].sum()
  print('pEarthquakeAlarm:', pEarthquakeAlarm)

  pCalledBurglary = (trace.posterior['john'].values[0]*trace.posterior['burglary'].values[0]+trace.posterior['mary'].values[0]*trace.posterior['burglary'].values[0]).sum()/trace.posterior['burglary'].values[0].sum()
  print('pCalledBurglary:', pCalledBurglary)



pAlarm: 23.220000000000006
pBurglaryAlarm: 0.2727272727272727
pEarthquakeAlarm: 0.2727272727272727
pCalledBurglary: 1.2857142857142858


### ODPOWIEDŹ:

YOUR ANSWER HERE

__UWAGA:__ Zwróć uwagę na wielkości podanych prawdopodobieńst aby dobarć odpowiednią liczbę symulacji.

&copy; Katedra Informatyki, Politechnika Krakowska